In [2]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [3]:
df = pd.read_csv(Path('./data/20200927.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,9/22/2020 18:11:07,A Moment of Innocence (1996),A Girl Walks Home Alone At Night (2014),A Dragon Arrives! (2016),JACKIE FUCKING CHAN,FROM A SMALL COUNTRY,IT’S A CLASSIC!,No Theme/Free-for-all
1,9/22/2020 18:12:58,A Dragon Arrives! (2016),A Girl Walks Home Alone At Night (2014),A Moment of Innocence (1996),FROM A SMALL COUNTRY,IT’S A CLASSIC!,JACKIE FUCKING CHAN,NaN
2,9/22/2020 18:14:00,A Dragon Arrives! (2016),A Girl Walks Home Alone At Night (2014),A Moment of Innocence (1996),IT’S A CLASSIC!,FROM A SMALL COUNTRY,JACKIE FUCKING CHAN,No Theme/Free-for-all
3,9/22/2020 18:38:24,A Girl Walks Home Alone At Night (2014),A Dragon Arrives! (2016),A Moment of Innocence (1996),IT’S A CLASSIC!,FROM A SMALL COUNTRY,No Theme/Free-for-all,JACKIE FUCKING CHAN
4,9/22/2020 19:02:45,A Dragon Arrives! (2016),A Moment of Innocence (1996),A Girl Walks Home Alone At Night (2014),JACKIE FUCKING CHAN,FROM A SMALL COUNTRY,IT’S A CLASSIC!,No Theme/Free-for-all
5,9/22/2020 22:14:24,A Girl Walks Home Alone At Night (2014),A Dragon Arrives! (2016),A Moment of Innocence (1996),IT’S A CLASSIC!,FROM A SMALL COUNTRY,JACKIE FUCKING CHAN,NaN
6,9/24/2020 11:02:16,A Girl Walks Home Alone At Night (2014),A Dragon Arrives! (2016),A Moment of Innocence (1996),IT’S A CLASSIC!,FROM A SMALL COUNTRY,JACKIE FUCKING CHAN,No Theme/Free-for-all


In [4]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [5]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['A Moment of Innocence (1996)',
       'A Girl Walks Home Alone At Night (2014)',
       'A Dragon Arrives! (2016)'], dtype=object)

In [6]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [7]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [8]:
submission_dict

{'A Moment of Innocence (1996)': <Candidate('A Moment of Innocence (1996)')>,
 'A Girl Walks Home Alone At Night (2014)': <Candidate('A Girl Walks Home Alone At Night (2014)')>,
 'A Dragon Arrives! (2016)': <Candidate('A Dragon Arrives! (2016)')>}

In [9]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [10]:
ballots

[<Ballot(A Moment of Innocence (1996), A Girl Walks Home Alone At Night (2014), A Dragon Arrives! (2016))>,
 <Ballot(A Dragon Arrives! (2016), A Girl Walks Home Alone At Night (2014), A Moment of Innocence (1996))>,
 <Ballot(A Dragon Arrives! (2016), A Girl Walks Home Alone At Night (2014), A Moment of Innocence (1996))>,
 <Ballot(A Girl Walks Home Alone At Night (2014), A Dragon Arrives! (2016), A Moment of Innocence (1996))>,
 <Ballot(A Dragon Arrives! (2016), A Moment of Innocence (1996), A Girl Walks Home Alone At Night (2014))>,
 <Ballot(A Girl Walks Home Alone At Night (2014), A Dragon Arrives! (2016), A Moment of Innocence (1996))>,
 <Ballot(A Girl Walks Home Alone At Night (2014), A Dragon Arrives! (2016), A Moment of Innocence (1996))>]

In [11]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [12]:
print(election_result)

ROUND 1
Candidate                                  Votes  Status
---------------------------------------  -------  --------
A Dragon Arrives! (2016)                       3  Hopeful
A Girl Walks Home Alone At Night (2014)        3  Hopeful
A Moment of Innocence (1996)                   1  Rejected

FINAL RESULT
Candidate                                  Votes  Status
---------------------------------------  -------  --------
A Girl Walks Home Alone At Night (2014)        4  Elected
A Dragon Arrives! (2016)                       3  Rejected
A Moment of Innocence (1996)                   0  Rejected



In [13]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                Votes  Status
---------------------  -------  --------
IT’S A CLASSIC!              4  Elected
JACKIE FUCKING CHAN          2  Rejected
FROM A SMALL COUNTRY         1  Rejected
No Theme/Free-for-all        0  Rejected

